In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import util

from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mpl.rcParams['figure.figsize'] = (16, 8)
mpl.rcParams['axes.grid'] = False

# Read Data

In [ ]:
df = pd.read_csv('cat9300_show_active_asic_cat9300_new.csv')

In [ ]:
df.head()

Remove rows with no device name column

In [ ]:
df = df[42222:]

In [ ]:
df.head()

In [ ]:
len(df)

Get Security Access Control Entries data

In [ ]:
sace_data = df[df['Desc'] == 'Security Access Control Entries']
ace = pd.DataFrame(sace_data, columns=['Time', 'Used1'])
ace['Time'] = pd.to_datetime(ace['Time'],unit='ns')

In [ ]:
ace.head()

In [ ]:
ace.tail()

In [ ]:
ace.reset_index(drop=True, inplace=True)
ace.head()

In [ ]:
ace.plot(y='Used1')

In [ ]:
len(ace['Used1'])

Lets visualize data having more variations

In [ ]:
ace.iloc[20000:50000].plot(y='Used1')

In [ ]:
small_data = ace.iloc[20000:50000]

In [ ]:
len(small_data)

# Prophet

Rename Datetime column as 'ds' and values column as 'y'

In [ ]:
small_data = small_data.rename(columns={'Time' : 'ds', 'Used1' : 'y'})

In [ ]:
small_data.head()

In [ ]:
small_data.tail()

In [ ]:
small_data.plot(y='y')

Create and train the model.

In prophet, the changepoint_prior_scale parameter is used to control how sensitive the trend is to changes, with a higher value being more sensitive and a lower value less sensitive.

In [ ]:
df_prophet = Prophet(changepoint_prior_scale=0.10)
df_prophet.fit(small_data)

To make forecasts, we need to create what is called a future dataframe. We specify the number of future periods to predict (5 days in our case) and the frequency of predictions (D - daily). We then make predictions with the prophet model we created and the future dataframe.

In [ ]:
forecast = df_prophet.make_future_dataframe(periods=5, freq='D')
prediction = df_prophet.predict(forecast)

Lets plot the prediction. The black dots represent the actual values, the blue line indicates the forecasted values, and the light blue shaded region is the uncertainty.

In [ ]:
df_prophet.plot(prediction, xlabel='Date', ylabel='Security Access Control Entries');

In [ ]:
df_prophet.plot_components(prediction);

# Predict for next 1 day 

Create and train the model.

In prophet, the changepoint_prior_scale parameter is used to control how sensitive the trend is to changes, with a higher value being more sensitive and a lower value less sensitive.

In [ ]:
df_prophet = Prophet()
df_prophet.fit(small_data)

To make forecasts, we need to create what is called a future dataframe. We specify the number of future periods to predict (5 days in our case) and the frequency of predictions (D - daily). We then make predictions with the prophet model we created and the future dataframe.

In [ ]:
forecast = df_prophet.make_future_dataframe(periods=1, freq='D')
prediction = df_prophet.predict(forecast)

Lets plot the prediction. The black dots represent the actual values, the blue line indicates the forecasted values, and the light blue shaded region is the uncertainty.

In [ ]:
df_prophet.plot(prediction, xlabel='Date', ylabel='Security Access Control Entries');

In [ ]:
df_prophet.plot_components(prediction);

In [ ]:
se = np.square(prediction.loc[:, 'yhat'] - small_data['y'])
mse = np.mean(se)
rmse = np.sqrt(mse)

In [ ]:
print("Mean Square Error:",mse, )
print("Root Mean Square Error: ", rmse)

The model accuracy is less but it has the feature to show daily, weekly, yearly trends

# LSTM

In [ ]:
small_data.head()

In [ ]:
small_data = small_data.rename(columns={'ds':'Time', 'y':'Used1'})

LSTMs are sensitive to the scale of the input data, specifically when the sigmoid or tanh activation functions are used. It’s generally a good practice to rescale the data to the range of [0, 1] or [-1, 1], also called normalizing. 

In [ ]:
values = small_data['Used1'].values.reshape(-1,1)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(values)

In [ ]:
scaled

Split data into train and test set

In [ ]:
train_size = int(len(scaled) * 0.8)
test_size = len(scaled) - train_size
train, test = scaled[:train_size, :], scaled[train_size:, :]
print(len(train), len(test))

In [ ]:
def create_dataset(dataset, look_back = 1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
        
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 2
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX, trainY

In [ ]:
testX, testY

In [ ]:
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

The LSTM network expects the input data to be provided with a specific array structure in the form of: [samples, time steps, features].
Our data is currently in the form [samples, features] and we are framing the problem as two time steps for each sample.

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
print(trainX.shape, testX.shape)

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
history = model.fit(trainX, trainY, epochs=20, batch_size=100, validation_data=(testX, testY), shuffle=False)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

Make Predictions

In [ ]:
train_predict = model.predict(trainX)
test_predict = model.predict(testX)

# invert predictions
train_predict = scaler.inverse_transform(train_predict)
Y_train = scaler.inverse_transform([trainY])
test_predict = scaler.inverse_transform(test_predict)
Y_test = scaler.inverse_transform([testY])

In [ ]:
print('Train Mean Absolute Error:', mean_absolute_error(Y_train[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_train[0], train_predict[:,0])))
print('Test Mean Absolute Error:', mean_absolute_error(Y_test[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(mean_squared_error(Y_test[0], test_predict[:,0])))

Compare Actual vs Prediction

In [ ]:
aa=[x for x in range(len(testY))]
plt.plot(aa, Y_test[0], marker='.', label="actual")
plt.plot(aa, test_predict[:,0], 'r', label="prediction")
plt.tight_layout()
plt.subplots_adjust(left=0.07)
plt.ylabel('Security ACE', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();

In [ ]:
aa=[x for x in range(500)]
plt.plot(aa, Y_test[0][3500:4000], marker='.', label="actual")
plt.plot(aa, test_predict[3500:4000,0], 'r', label="prediction")
plt.tight_layout()
plt.subplots_adjust(left=0.07)
plt.ylabel('Security ACE', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();